### CrewAI Framework Documentation: https://docs.crewai.com/how-to/LLM-Connections/

# MAKE THE ENVIRONMENT READY 

In [1]:
#!pip install crewai crewai_tools langchain_community 

In [2]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os

os.environ["OPENAI_API_KEY"] = 'YOUR API KEY'
openai_key=os.environ.get('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Google search get your API key here: https://serpapi.com/
os.environ["SERPER_API_KEY"] = 'YOUR API KEY'

# CrewAI Tools

In [5]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools (Google Serper Search + Scraper)
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# CREATING AGENTS
## Define your agents and provide them a role, goal and backstory.

In [6]:
financial_analyst_agent = Agent(
    role="Financial Analyst",
    goal="Analyze the start-up in terms of financials and investment stage",
    backstory="""Specializing in financial markets, this agent 
              uses research to identify current investors in the startup
              and understanding what stage of investment it is including 
              current revenue and future revenue projections.""",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
competition_analyst_agent = Agent(
    role="Competition Analyst",
    goal="Identify current incumbants or direct competition to the start-up and compare and contrast",
    backstory="""Equipped with a deep understanding of financial 
              markets and quantitative analysis, this agent
              benchmarks startup against its key competitors. It evaluates 
              the SWOT (strenghts-weaknesses-opportunities-threats) analysis to determine 
              the most profitable and risk-averse options.""",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential investment.",
    backstory="""Armed with a deep understanding of risk assessment models 
              and market dynamics, this agent scrutinizes the potential 
              risks of proposed investment. It offers a detailed analysis of 
              risk exposure and compares it to my risk tolerance.""",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
recommender_agent = Agent(
    role="Recommendation Advisor",
    goal="Evaluate whether to invest in a startup or not and if so is it wothwhile to invest as a small investor.",
    backstory="""Armed with a deep understanding of risk assessment models 
              and market dynamics, this agent scrutinizes the potential 
              risks of proposed trades. It offers a detailed recommendation of why to invest or not.""",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Create Tasks

In [11]:
financial_analysis_task = Task(
    description=(
        """Analyze financial market data for 
        the selected start-up ({startup}). 
        Understand creditbility of current investors in ({startup}).
        Use sources like Yahoo Finance, TechCrunch and similar websites.
        The company website is ({website})."""
    ),
    expected_output=(
        """Insights about their current investors and their credibility.
        Financials of the {startup} in terms of current investment to date as well as including how they make money (revenue)
        and their monetization strategy.
        List their current revenue and future projected revenue."""
    ),
    agent=financial_analyst_agent,
    async_execution=True
)

In [12]:
competition_analysis_task = Task(
    description=(
        """Analyze competition for 
        the selected start-up ({startup}). 
        Use statistical modeling and machine learning to 
        identify trends and predict market movements.
        The company website is ({website})."""
    ),
    expected_output=(
        "Insights and alerts about competition in terms "
        "opportunities or threats for {startup}."
    ),
    agent=competition_analyst_agent,
    async_execution=True
)

In [13]:
risk_assessment_task = Task(
    description=(
        """Evaluate the risks associated with the proposed investment 
        strategies and execution plans for {startup}. 
        Provide a detailed analysis of potential risks.
        Identify any risk raised in specialized forums in Reddit as well.
        The company website is ({website})."""
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks for {startup}."
    ),
    context=[financial_analysis_task, competition_analysis_task],
    agent=risk_management_agent
)

In [14]:
recommender_task = Task(
    description=(
        "Analyze all data"
        "the selected start-up ({startup}). "
        "The company website is ({website})."
    ),
    expected_output=(
        """Make a detailed recommendation on why or why not to invest in {startup}.
        Suggest if there is a better alternative available from the identified competitors.
        Include summarized output in terms of current investors, current investment amount, current and projected revenue,
        key competitors."""
    ),
    context=[financial_analysis_task, competition_analysis_task, risk_assessment_task],
    agent=recommender_agent 
)

# Creating the crew
## Pass the tasks to be performed by those agents.

In [15]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[financial_analyst_agent, 
            competition_analyst_agent, 
            risk_management_agent, 
            recommender_agent],
    
    tasks=[financial_analysis_task, 
           competition_analysis_task, 
           risk_assessment_task, 
          recommender_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.2),
    process=Process.hierarchical,
    verbose=True
)

## Running the crew

In [16]:
# Example data for kicking off the process
financial_trading_inputs = {
    'startup': 'CrewAI',
    'website': 'CrewAI.com',
    'risk_tolerance': 'Medium',
    'news_impact_consideration': True
}

In [17]:
# this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Crew Manager
## Task: Analyze financial market data for 
        the selected start-up (CrewAI). 
        Understand creditbility of current investors in (CrewAI).
        Use sources like Yahoo Finance, TechCrunch and similar websites.
        The company website is (CrewAI.com).
# Agent: Crew Manager
## Task: Analyze competition for 
        the selected start-up (CrewAI). 
        Use statistical modeling and machine learning to 
        identify trends and predict market movements.
        The company website is (CrewAI.com).
# Agent: Financial Analyst
## Task: Research and compile information about CrewAI, including their current investors, financials, revenue, and monetization strategy
# Agent: Competition Analyst
## Task: Analyze competition for CrewAI using statistical modeling and machine learning to identify trends and predict market movements. Provide insights and alerts about competition in terms of opportunities or threats for CrewAI.


# Agent: Competition Analys

In [18]:
print(result)

After conducting a thorough analysis of CrewAI and its competitors, it is recommended to consider investing in CrewAI. The company has established partnerships with industry leaders like IBM and Fortune 500 companies, offering innovative solutions for automated workflows powered by AI technology. Despite facing internal and external risks, CrewAI's strategic positioning and focus on AI-driven automation present a promising opportunity for investment. It is advised to delve deeper into financial metrics, market trends, and long-term growth strategies to make an informed decision. Additionally, exploring alternative investment opportunities among identified competitors like Lyzr, Dify.AI, and AgentOps could provide a diversified portfolio approach in the AI automation sector.
